In [ ]:
import cartopy.crs as ccrs
import matplotlib.pylab as plt
import numpy as np
import xarray as xr
import cmaps
import sacpy as scp
from cartopy.util import add_cyclic_point
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm
import sys
sys.path.append('/public/home/songqh/Documents/python_packages/')
from sqh_toolbox import plot_figure
import warnings
warnings.filterwarnings("ignore") 
plt.rcParams["font.family"] = "Liberation Sans"
plt.rcParams["font.weight"] = "bold"

In [ ]:
file1 = xr.open_dataset("/public/home/songqh/my_data/new_ERA5/monthly/ERA5_precipitation_1940-2024.nc")
precip = file1['tp'].loc[file1.time.dt.year.isin(np.arange(1979,2025,1))&file1.time.dt.month.isin([6,7,8]),40:0,60:100].groupby('time.year').mean('time').rename({'year':'time'})*1000/30
precip_mean = precip.mean('time')
lon1 = precip.longitude.loc[60:100]
lat1 = precip.latitude.loc[40:0]

In [ ]:
file1 = xr.open_dataset("/public/home/songqh/my_data/new_ERA5/monthly/ERA5_precipitation_1940-2024.nc")
precip_index = file1['tp'].loc[file1.time.dt.year.isin(np.arange(1979,2025,1))&file1.time.dt.month.isin([6,7,8]),27:10,70:100].groupby('time.year').mean('time').rename({'year':'time'}).mean(('longitude','latitude'))*1000
precip_index = (precip_index - np.mean(precip_index,axis=0))/np.std(precip_index)

In [ ]:
rainfall = np.zeros((41,41))
rainfall[12:35,3:40]=precip_mean[12:35,3:40]

In [ ]:
gph = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/gph_200hPa_1979-2024.npy")
t2m = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/t2m_1979-2024.npy")
lon = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/lon.npy")
lat = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/lat.npy")

In [ ]:
LinReg1 = scp.LinReg(precip_index[:],gph[:]).slope
LinReg2 = scp.LinReg(precip_index[:],t2m[:]).slope
LinReg2[:140,:] = 0

In [ ]:
ncl_cmap = cm.get_cmap(cmaps.CBR_wet, 256)  
cmap_colors = ncl_cmap(np.linspace(0, 1, 256))  
cmap_colors = cmap_colors[30:]
modified_CBR_wet = LinearSegmentedColormap.from_list("modified_CBR_wet", cmap_colors)

In [ ]:
cycle_slope,cycle_lon = add_cyclic_point(LinReg2, coord=lon)
plt.figure(figsize=(12,12))
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=90, central_latitude=-40))
ax.coastlines(resolution='50m',color='k',zorder=4,lw=1.2)
ax.contourf(lon1,lat1,rainfall,zorder=3,levels=np.linspace(0,1,11),
    transform=ccrs.PlateCarree(), cmap=modified_CBR_wet)
ax.contour(lon,lat,LinReg1/9.8,colors='k',linewidths=1.8,levels=np.linspace(-30,30,15), zorder=3,transform=ccrs.PlateCarree())
ax.contourf(cycle_lon,lat,cycle_slope, zorder=2,levels=np.linspace(-1,1,11),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=cmaps.BlueWhiteOrangeRed)
plt.savefig("/public/home/songqh/project/ISM_impact_Antarctic/figures/response_figures/diagram.jpeg",dpi=600, bbox_inches='tight')